In [1]:
# baseline cnn model for digit
import numpy as np
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization



In [2]:
#loading the dataset
input = np.loadtxt("mfeat-pix.txt", dtype='i')
#Converting the dataset into a 2D matrix
input_2D = np.reshape(input,(2000,16,15))

In [3]:
def dataset():
# Dividing the data into train and test data
    testX = np.zeros((400,16,15))
    trainX = np.zeros((1600,16,15))
    testY = np.zeros((400,1), dtype = 'uint')
    trainY = np.zeros((1600,1), dtype = 'uint')

    for i in range(10):
        # Taking last 40 examples per example for test and 16 for train
        testX[(i*40):((i+1)*40)] = input_2D[(200*i)+159:(200*i+199)]
        trainX[(i*160):((i+1)*160-1)] = input_2D[(200*i):(200*i)+159]
        # Labels
        testY[(i*40):((i+1)*40)] = i
        trainY[(i*160):((i+1)*160-1)] = i
    trainX = trainX.reshape((trainX.shape[0], 16, 15, 1))
    testX = testX.reshape((testX.shape[0], 16, 15, 1))
    testY = to_categorical(testY)
    trainY = to_categorical(trainY)
    return trainX, trainY, testX, testY



In [4]:
#Data normalization
def normalise_data(traindata, testdata):
    traindata = traindata.astype('float32')
    testdata = testdata.astype('float32')
    train_norm = traindata / 6.0
    test_norm = testdata / 6.0
    return train_norm, test_norm

In [5]:
def model_definition():
    model = Sequential()
    model.add(Conv2D(16, (2, 2), activation='relu', kernel_initializer='he_uniform', input_shape=(16, 15, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    # softmax for multimodal classification
    model.add(Dense(10, activation='softmax'))
    opt = SGD(lr=0.005)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def train_model(mtrainX, mtrainY, folds=5):
    accuracies = list()
    # random shuffle k fold validation
    kfold = KFold(folds, shuffle=True, random_state=3)
    for training_indices, testing_indices in kfold.split(mtrainX):
        model = model_definition()
        trainX, trainY, testX, testY = mtrainX[training_indices], mtrainY[training_indices], mtrainX[testing_indices], mtrainY[testing_indices]
        model.fit(trainX, trainY, epochs=50, batch_size=8, validation_data=(testX, testY), verbose=0)
        _, accuraccy = model.evaluate(testX, testY, verbose=0)
        print('%.3f' % (accuraccy * 100.0))
        # stores accuracies and test data on last kth model
        accuracies.append(accuraccy)
    return accuracies, model

In [7]:
def run_model():
    trainX, trainY, testX, testY = dataset()
    trainX, testX = normalise_data(trainX, testX)
    # training model
    accuracies,model = train_model(trainX, trainY)
    # Test model on kth trained model
    _, accuracy1 = model.evaluate(testX, testY, verbose=0)
    print('Test accuracy =',accuracy1*100.0)
    return accuracy1


In [8]:
run_model()

98.125
97.188
96.875
97.188
96.250
Test accuracy = 98.25000166893005


0.9825000166893005